In [4]:
import google.generativeai as genai
import json
import time
import pandas as pd

/Users/yadder/miniconda3/envs/project_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [5]:
def get_gemini_api_key(config_path='gemini.config'):
    try:
        with open(config_path, 'r') as f:
            for line in f:
                if line.startswith('API_KEY='):
                    return line.split('=')[1].strip()
    except FileNotFoundError:
        print(f"Error: Config file not found at {config_path}")
        print("Please create a gemini.config file with your API key in the format:")
        print("API_KEY=your_api_key_here")
    except Exception as e:
        print(f"Error reading config file: {e}")
    
    return None

### Ask Gemini

In [6]:
def ask_gemini(batch):
    prompt = f"""
    Act as an expert in zoology. I need technical information about these animals. If it's an plant don't include it in the JSON: {batch}.
    
    For each animal, return a JSON object with the following exact fields:
    - "scientific_name": Name of the animal (the same scientific name that I am using as input).
    - "common_name"
    - "weight": Approximate weight in kg (only number and unit).
    - "size": Height or length in meters (only number and unit).
    - "diet": Carnivore, Herbivore, Omnivore, Insectivore.
    - "life_span": Life span in years (e.g. average number of years or range like "10-15 years").
    - "habitat": 5 habitats of the animal as maximum (e.g. "forest, grassland, etc."). Just one word for each habitat.
    - "continent": Continent where the animal is found (e.g. "Asia, Africa, Europe, South America, North America, Central America, Oceania, Antarctica").

    Return a list of JSON objects.
    """
    
    try:
        response = model.generate_content(prompt)
        return json.loads(response.text)
    except Exception as e:
        print(f"Error requesting batch: {e}")
        return []

## Features Extraction

In [10]:
START_RECORD = 5000
END_RECORD = 6000
BATCH_SIZE = 10

In [11]:
API_KEY = get_gemini_api_key()

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash',
                              generation_config={"response_mime_type": "application/json"})

# Read the CSV file
df = pd.read_csv('../data/animals.csv')  # Update with your file path if needed

# Get all values from a specific column as a list
animals = df['scientific_name'].tolist()

print(f"Starting with {len(animals)} animals...")

# Initialize results list
total_response = []


for i in range(START_RECORD, END_RECORD, BATCH_SIZE):
    
    if i >= len(animals):
        break

    upper_bound = min(len(animals) - 1, i + BATCH_SIZE)
    current_batch = animals[i:upper_bound]
    
    print(f"Requesting batch {i//BATCH_SIZE + 1}: {current_batch}...")
    
    # Ask Gemini
    gemini_response = ask_gemini(current_batch)
    
    # Guardar resultados
    total_response.extend(gemini_response)
    
    # (Opcional) Pequeña pausa para no saturar si tienes miles de animales
    time.sleep(1) 

# Guardar resultados en un archivo JSON
with open(f"../data/gemini_animal_features_{START_RECORD}_{END_RECORD}.json", "w") as f:
    json.dump(total_response, f, indent=4)

print(f"Processed {len(total_response)} records and saved to gemini_animal_features_{START_RECORD}_{END_RECORD}.json")

Starting with 5159 animals...
Requesting batch 501: ['Vanessa virginiensis', 'Vangueria infausta infausta', 'Vangueria madagascariensis', 'Vangueria vestita', 'Varanus bengalensis', 'Varanus niloticus', 'Varanus salvator macromaculatus', 'Velarifictorus micado', 'Velella velella', 'Veneridae']...
Requesting batch 502: ['Ventridens', 'Vepris lanceolata', 'Verbena litoralis', 'Verbesina alternifolia', 'Verbesina encelioides', 'Vernonia colorata', 'Vertebrata', 'Vespa affinis', 'Vespa crabro', 'Vespa orientalis']...
Requesting batch 503: ['Vespa velutina', 'Vespa velutina nigrithorax', 'Vespericola columbianus', 'Vespidae', 'Vespinae', 'Vespula', 'Vespula alascensis', 'Vespula atropilosa', 'Vespula flavopilosa', 'Vespula germanica']...
Requesting batch 504: ['Vespula maculifrons', 'Vespula pensylvanica', 'Vespula squamosa', 'Vespula vulgaris', 'Viburnum opulus', 'Viburnum recognitum', 'Vicia cracca', 'Vicia grandiflora', 'Vigna unguiculata', 'Villa']...
Requesting batch 505: ['Vindula ero